# Week 7: Localization and Mapping

### Topics Covered

- Global Navigation Satellite Systems (GNSS/GPS); Simultaneous Localization and Mapping (SLAM); Particle Filters for Global Localization; High-Definition (HD) Map representation

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand the key concepts
2. Implement algorithms
3. Apply techniques to real-world problems

---

## Setup

Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Ellipse
from matplotlib.animation import FuncAnimation
from scipy.stats import multivariate_normal
from collections import defaultdict

# Set random seed for reproducibility
np.random.seed(42)

## 1. Global Navigation Satellite Systems (GNSS/GPS)

GNSS provides global position estimates by triangulating signals from satellites. GPS is the most widely known GNSS operated by the United States.

### How GPS Works

**Trilateration Principle:**
- Satellites continuously broadcast their position and time
- GPS receiver measures time delay to calculate distance to each satellite
- Need at least 4 satellites for 3D position (x, y, z) + clock bias

**Position Calculation:**

Distance to satellite $i$:
$$\rho_i = c \cdot \Delta t_i$$

Where:
- $c$ = speed of light (≈ 3 × 10⁸ m/s)
- $\Delta t_i$ = time delay from satellite $i$

**Trilateration Equations:**
$$\sqrt{(x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2} = \rho_i$$

For each satellite $i \in \{1, 2, 3, 4\}$.

### Error Sources

1. **Atmospheric Delays**
   - Ionospheric delay: ±5-10 meters
   - Tropospheric delay: ±0.5-5 meters

2. **Multipath Effects**
   - Signal reflections from buildings/terrain
   - Urban canyons: ±10-50 meters

3. **Satellite Geometry (DOP - Dilution of Precision)**
   - Poor geometry → large errors
   - GDOP (Geometric DOP), HDOP (Horizontal), VDOP (Vertical)

4. **Clock Errors**
   - Satellite clock drift
   - Receiver clock bias

5. **Selective Availability** (historical, now disabled)

### GPS Accuracy Levels

| System | Accuracy | Notes |
|--------|----------|-------|
| Standard GPS | ±5-10m | Consumer devices |
| DGPS (Differential) | ±1-3m | Ground-based corrections |
| RTK GPS | ±2cm | Real-time kinematic, requires base station |
| PPP (Precise Point) | ±10cm | Post-processing |

### Limitations for Autonomous Vehicles

- **Update Rate**: 1-10 Hz (too slow for high-speed control)
- **Latency**: 100-200ms
- **Availability**: Poor in tunnels, parking garages, urban canyons
- **Accuracy**: Insufficient for lane-level localization (need ±10cm)

**Solution**: Fuse GPS with other sensors (IMU, odometry, cameras, LIDAR) using Kalman Filters or Particle Filters.

In [ ]:
# Simple HD Map representation and localization simulation

class HDMap:
    def __init__(self):
        """Simple HD Map with lane boundaries and landmarks"""
        # Create a curved road - compute all points first
        s_values = np.linspace(0, 50, 100)
        
        # Centerline (curved path)
        x_values = s_values
        y_values = 5 * np.sin(s_values / 10)
        
        # Compute heading from gradient
        dx = np.gradient(x_values)
        dy = np.gradient(y_values)
        headings = np.arctan2(dy, dx)
        
        # Lane boundaries (offset from centerline)
        lane_width = 3.5
        
        # Left boundary
        x_left = x_values - lane_width/2 * np.sin(headings)
        y_left = y_values + lane_width/2 * np.cos(headings)
        
        # Right boundary
        x_right = x_values + lane_width/2 * np.sin(headings)
        y_right = y_values - lane_width/2 * np.cos(headings)
        
        self.road_centerline = np.column_stack([x_values, y_values])
        self.lane_left = np.column_stack([x_left, y_left])
        self.lane_right = np.column_stack([x_right, y_right])
        
        # Add some landmark features (poles, signs)
        self.landmarks = np.array([
            [10, 8],
            [20, 7],
            [30, 8],
            [40, 7]
        ])
    
    def get_closest_point_on_centerline(self, position):
        """Find closest point on road centerline"""
        distances = np.linalg.norm(self.road_centerline - position, axis=1)
        idx = np.argmin(distances)
        return self.road_centerline[idx], idx
    
    def visualize(self, ax):
        """Draw HD map on axis"""
        ax.plot(self.road_centerline[:, 0], self.road_centerline[:, 1], 
               'k--', linewidth=1, label='Centerline')
        ax.plot(self.lane_left[:, 0], self.lane_left[:, 1], 
               'y-', linewidth=2, label='Lane Boundaries')
        ax.plot(self.lane_right[:, 0], self.lane_right[:, 1], 'y-', linewidth=2)
        ax.scatter(self.landmarks[:, 0], self.landmarks[:, 1], 
                  c='red', s=100, marker='^', label='Landmarks', zorder=10)


def demonstrate_hd_map_localization():
    """Demonstrate localization using HD map matching"""
    
    # Create HD map
    hd_map = HDMap()
    
    # Simulate vehicle trajectory (with GPS noise)
    true_trajectory = []
    noisy_gps_trajectory = []
    
    for i in range(0, 80, 2):
        # True position (on centerline)
        true_pos = hd_map.road_centerline[i]
        true_trajectory.append(true_pos)
        
        # Noisy GPS
        gps_pos = true_pos + np.random.randn(2) * 3  # ±3m GPS noise
        noisy_gps_trajectory.append(gps_pos)
    
    true_trajectory = np.array(true_trajectory)
    noisy_gps_trajectory = np.array(noisy_gps_trajectory)
    
    # Map-matched trajectory (project GPS onto road)
    map_matched_trajectory = []
    
    for gps_pos in noisy_gps_trajectory:
        closest_point, _ = hd_map.get_closest_point_on_centerline(gps_pos)
        map_matched_trajectory.append(closest_point)
    
    map_matched_trajectory = np.array(map_matched_trajectory)
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Left: Map with trajectories
    ax = axes[0]
    hd_map.visualize(ax)
    
    ax.plot(true_trajectory[:, 0], true_trajectory[:, 1], 
           'g-', linewidth=3, label='True Position', zorder=5)
    ax.scatter(noisy_gps_trajectory[:, 0], noisy_gps_trajectory[:, 1], 
              c='red', s=30, alpha=0.5, label='Noisy GPS', zorder=3)
    ax.plot(map_matched_trajectory[:, 0], map_matched_trajectory[:, 1], 
           'b-', linewidth=2, label='Map-Matched', zorder=4)
    
    ax.set_xlabel('X Position (m)')
    ax.set_ylabel('Y Position (m)')
    ax.set_title('HD Map Localization')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axis('equal')
    
    # Right: Error comparison
    ax2 = axes[1]
    
    gps_error = np.linalg.norm(noisy_gps_trajectory - true_trajectory, axis=1)
    map_matched_error = np.linalg.norm(map_matched_trajectory - true_trajectory, axis=1)
    
    steps = np.arange(len(gps_error))
    ax2.plot(steps, gps_error, 'r-', linewidth=2, label='GPS Error', alpha=0.7)
    ax2.plot(steps, map_matched_error, 'b-', linewidth=2, label='Map-Matched Error')
    ax2.axhline(np.mean(gps_error), color='r', linestyle='--', alpha=0.5,
               label=f'Mean GPS: {np.mean(gps_error):.2f}m')
    ax2.axhline(np.mean(map_matched_error), color='b', linestyle='--', alpha=0.5,
               label=f'Mean Matched: {np.mean(map_matched_error):.2f}m')
    
    ax2.set_xlabel('Time Step')
    ax2.set_ylabel('Position Error (m)')
    ax2.set_title('Localization Accuracy Comparison')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print(f"GPS Mean Error: {np.mean(gps_error):.2f} m")
    print(f"Map-Matched Mean Error: {np.mean(map_matched_error):.2f} m")
    print(f"Improvement: {(1 - np.mean(map_matched_error)/np.mean(gps_error))*100:.1f}%")

demonstrate_hd_map_localization()

## Exercises

### Exercise 1: Particle Filter for Kidnapped Robot Problem

Implement a particle filter that can recover when the robot is "kidnapped" (teleported to an unknown location).

**Tasks:**
1. Modify the particle filter to occasionally inject random particles
2. Simulate kidnapping the robot at step 5
3. Show how the filter recovers
4. Compare recovery time with different numbers of random particles injected

**Hint:** Add uniform random particles when effective sample size drops below a threshold.

In [ ]:
# Exercise 3 - Starter code

def create_multi_lane_map():
    """Create HD map with 3 lanes"""
    # TODO: Define lane centerlines
    # Lane 1: y = 0
    # Lane 2: y = 3.5
    # Lane 3: y = 7.0
    pass

def lane_level_localization():
    """Determine which lane the vehicle is in"""
    # TODO:
    # 1. Get GPS position (noisy)
    # 2. Detect lane markings (lateral offset)
    # 3. Fuse to determine lane ID
    # 4. Track lane changes
    pass

# Your implementation here

## References and Additional Resources

### Core Textbooks

1. **Probabilistic Robotics** by Thrun, Burgard, and Fox (2005)
   - Chapter 8: Mobile Robot Localization (Monte Carlo Localization)
   - Chapter 10: SLAM
   - Chapter 13: FastSLAM

2. **State Estimation for Robotics** by Barfoot (2017)
   - Chapters on SLAM and mapping

3. **Autonomous Driving** by Yurtsever et al. (2020)
   - Survey on localization methods for autonomous vehicles

### Key Papers

1. **Monte Carlo Localization**
   - Dellaert et al. (1999) - "Monte Carlo Localization for Mobile Robots"
   - Fox et al. (1999) - "Monte Carlo Localization: Efficient Position Estimation"

2. **SLAM**
   - Dissanayake et al. (2001) - "A Solution to the SLAM Problem"
   - Durrant-Whyte & Bailey (2006) - "Simultaneous Localization and Mapping: Part I & II"
   - Cadena et al. (2016) - "Past, Present, and Future of SLAM" (Survey)

3. **Visual SLAM**
   - Mur-Artal & Tardós (2017) - "ORB-SLAM2: an Open-Source SLAM System"
   - Campos et al. (2021) - "ORB-SLAM3"

4. **HD Maps**
   - Seif & Hu (2016) - "Autonomous Driving in the iCity - HD Maps as a Key Challenge"
   - Liu et al. (2020) - "High Definition Maps for Autonomous Driving"

### Software & Libraries

1. **SLAM Libraries**
   - **g2o**: General Graph Optimization - https://github.com/RainerKuemmerle/g2o
   - **GTSAM**: Georgia Tech Smoothing and Mapping - https://github.com/borglab/gtsam
   - **ORB-SLAM3**: Visual SLAM - https://github.com/UZ-SLAMLab/ORB_SLAM3
   - **Cartographer**: Google's SLAM - https://github.com/cartographer-project/cartographer

2. **Particle Filter**
   - **particle-filter-tutorial**: Python implementations
   - ROS packages: amcl (Adaptive Monte Carlo Localization)

3. **HD Maps**
   - **Lanelet2**: https://github.com/fzi-forschungszentrum-informatik/Lanelet2
   - **OpenDRIVE**: Industry standard format
   - **Apollo HD Map**: Baidu's open-source HD map module

### Online Resources

1. **Cyrill Stachniss YouTube Lectures**
   - Mobile Sensing and Robotics course
   - Excellent SLAM and localization lectures

2. **Self-Driving Cars Specialization (Coursera)**
   - University of Toronto
   - Modules on localization and mapping

3. **Udacity Self-Driving Car Nanodegree**
   - Particle filter project
   - Kidnapped vehicle project

### Datasets

1. **KITTI Dataset**: Autonomous driving benchmark
2. **nuScenes**: Full sensor suite with HD maps
3. **Waymo Open Dataset**: Large-scale AV dataset
4. **Oxford RobotCar**: Long-term autonomy dataset

### Industry Standards

- **ISO 19157**: Geographic Information - Data Quality
- **OpenDRIVE**: Road network description
- **NDS (Navigation Data Standard)**: Automotive navigation
- **Lanelet/Lanelet2**: Autonomous driving maps

### Applications in Autonomous Vehicles

1. **Localization Stack**
   - GPS/GNSS: Coarse global position
   - IMU: High-rate pose updates
   - Wheel odometry: Dead reckoning
   - LIDAR: Map matching and obstacle detection
   - Camera: Lane detection and visual odometry

2. **Map Update Strategies**
   - Crowdsourced mapping (fleet learning)
   - Real-time dynamic layer updates
   - Cloud-based map management

3. **Challenges**
   - Urban canyons (GPS multipath)
   - Tunnels (no GPS)
   - Dynamic environments
   - Map scalability and storage

---

## Summary

This notebook covered localization and mapping for autonomous vehicles:

**Key Concepts:**
- GPS/GNSS principles and error sources
- Particle Filters for global localization
- SLAM (Simultaneous Localization and Mapping)
- HD Maps for precise localization

**Practical Skills:**
- Implementing particle filters with resampling
- Building EKF-SLAM systems
- Map-matching for improved accuracy
- Multi-sensor fusion strategies

**Next Steps:**
1. Implement a full sensor fusion system
2. Explore Graph-SLAM and optimization-based approaches
3. Study Visual-Inertial Odometry (VIO)
4. Learn about semantic SLAM and dynamic environments